In [1]:
from astropy import units as u
from gala.units import UnitSystem
usys = UnitSystem(u.kpc, u.Myr, u.Msun, u.radian)
import jax.numpy as jnp
import streamsculptor as ssc
import interpax
import jax
from astropy.constants import G

In this example we convert a density profile to a potential object. This is accomplished by solving Poisson's equation over a grid, and storing the solution using 1d cubic interpolation.
We will use the Zhao density as an example: http://ui.adsabs.harvard.edu/abs/1996MNRAS.278..488Z/abstract

In [3]:
@jax.jit
def zhao_density(m, r_s, alpha, beta, gamma, r):
    uu = r / r_s
    rho0 = m / (4 * jnp.pi * r_s**3)

    b = (beta - gamma) * alpha
    return rho0 / uu**gamma / (1.0 + uu ** (1.0 / alpha)) ** b


The density function must have a single argument: the 1D spherical radius, r

In [4]:
m = 1e11
r_s = 10.0
alpha = 1.0
beta = 2.5
gamma = 1.0

density_func = lambda r: zhao_density(m, r_s, alpha, beta, gamma, r)


In [5]:
from streamsculptor.potential import get_potential_from_density
zhao_profile = get_potential_from_density(density_func, units=usys)

That's it! You can now use `zhao_profile` like any other `streamsculptor` potential object

In [6]:
# Evaluate the potential
zhao_profile.potential(jnp.array([1.,2.,3.]),0.0)

Array(-0.08283585, dtype=float64)

In [7]:
# Evaluate the gradient
zhao_profile.gradient(jnp.array([1.,2.,3.]),0.0)

Array([0.0004346, 0.0008692, 0.0013038], dtype=float64)

Below, we illustrate `get_potential_from_density` using the Plummer profile which has a simple analytic density/potential pairing

In [11]:
plummer = ssc.potential.PlummerPotential(m=1e11, r_s=5.0, units=usys)
plummer_density = lambda r: plummer.density(jnp.array([r,0,0]),0.0)
plummer_test = get_potential_from_density(plummer_density, units=usys)


In [20]:
# Test it out
xyz_test  = jnp.array([.1,20.,30.])
print("potential from density test: " + str(jnp.allclose(plummer.potential(xyz_test,0.0), plummer_test.potential(xyz_test,0.0), rtol=1e-3)))

potential from density test: True


In [24]:
print("acceleration from density test: " + str(jnp.allclose(plummer.acceleration(xyz_test,0.0), plummer_test.acceleration(xyz_test,0.0), rtol=1e-3)))

acceleration from density test: True
